### Import some useful libraries

In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn import ensemble,model_selection,preprocessing,feature_selection,metrics
import xgboost as xgb
import lightgbm as lgb
import scipy.stats as st

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Import the files as dataframes

In [2]:
df_air_reserve = pd.read_csv('air_reserve.csv.zip')
df_air_store = pd.read_csv('air_store_info.csv.zip')
df_air_visit = pd.read_csv('air_visit_data.csv.zip')
df_hpg_reserve = pd.read_csv('hpg_reserve.csv.zip')
df_hpg_store = pd.read_csv('hpg_store_info.csv.zip')
df_date_info = pd.read_csv('date_info.csv.zip')
df_store_id_rel = pd.read_csv('store_id_relation.csv.zip')
df_sample = pd.read_csv('sample_submission.csv.zip')

### Some constants for later

In [3]:
# map months to seasons
seasons = {'Jan': 'Winter','Feb': 'Winter','Mar': 'Spring','Apr': 'Spring','May': 'Spring','Jun': 'Summer',
           'Jul': 'Summer','Aug': 'Summer','Sep': 'Autumn','Oct': 'Autumn','Nov': 'Autumn','Dec': 'Winter'}

# roll up genres
genres = {
    'Japanese style':'Japanese',
    'International cuisine':'Other',
    'Grilled meat':'Other Asian',
    'Creation':'Japanese',
    'Italian':'European',
    'Seafood':'Other',
    'Spain Bar/Italian Bar':'European',
    'Japanese food in general':'Japanese',
    'Shabu-shabu/Sukiyaki':'Japanese',
    'Chinese general':'Other Asian',
    'Creative Japanese food':'Japanese',
    'Japanese cuisine/Kaiseki':'Japanese',
    'Korean cuisine':'Other Asian',
    'Okonomiyaki/Monja/Teppanyaki':'Japanese',
    'Karaoke':'Bar or Club',
    'Steak/Hamburger/Curry':'Other',
    'French':'European',
    'Cafe':'European',
    'Bistro':'Other',
    'Sushi':'Japanese',
    'Party':'Bar or Club',
    'Western food':'Other',
    'Pasta/Pizza':'Other',
    'Thai/Vietnamese food':'Other Asian',
    'Bar/Cocktail':'Bar or Club',
    'Amusement bar':'Bar or Club',
    'Cantonese food':'Other Asian',
    'Dim Sum/Dumplings':'Other Asian',
    'Sichuan food':'Other Asian',
    'Sweets':'Other',
    'Spain/Mediterranean cuisine':'European',
    'Udon/Soba':'Japanese',
    'Shanghai food':'Other Asian',
    'Taiwanese/Hong Kong cuisine':'Other Asian',
    'Japanese food':'Japanese', 
    'Dining bar':'Bar or Club', 
    'Izakaya':'Japanese',
    'Okonomiyaki/Monja/Teppanyaki':'Japanese', 
    'Italian/French':'European', 
    'Cafe/Sweets':'Other',
    'Yakiniku/Korean food':'Other Asian', 
    'Western food':'Other', 
    'Bar/Cocktail':'Bar or Club', 
    'Other':'Other',
    'Creative cuisine':'Japanese', 
    'Karaoke/Party':'Bar or Club', 
    'International cuisine':'Other',
    'Asian':'Other Asian',
    'None':'None',
    'No Data':'No Data'}

# function to aggregate various values to add as features. Tables will be added to a dictionary
def groupings(df,group,field,agg,name,cols):
    for i in group:
        tmp = df.groupby([i])\
                .agg(agg)
        tmp.columns = tmp.columns.map(''.join)
        tmp.columns=[s + i for s in cols]
        tmp.reset_index(inplace=True)
        name[i]=tmp
        name[i].columns = name[i].columns.map(''.join)
        for j in field:
            tmp = df.groupby([i,j])\
                    .agg(agg)
            tmp.columns = tmp.columns.map(''.join)
            tmp.columns=[s + i+j for s in cols]
            tmp.reset_index(inplace=True)
            name[i+j] = tmp
            
# for the CV
splits=3
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

# calculate error
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

### Wrangle the reservation file

In [4]:
# merge 'air' tables and bring over any 'hpg' store data
df_air_merged = df_air_reserve.merge(df_air_store,on='air_store_id', how='left').merge(
    df_store_id_rel, on='air_store_id', how='left').merge(df_hpg_store,on='hpg_store_id', how='left',suffixes=('_air','_hpg'))

In [5]:
# merge 'hpg' tables and bring over any 'air' store data
df_hpg_merged = df_hpg_reserve.merge(df_hpg_store,on='hpg_store_id', how='left').merge(
    df_store_id_rel,on='hpg_store_id', how='left').merge(df_air_store,on='air_store_id', how='left',suffixes=('_hpg','_air'))

In [6]:
# add source column
df_air_merged['source'] = 'air'
df_hpg_merged['source'] = 'hpg'

In [7]:
# append tables together
df_res_merged = df_air_merged.append(df_hpg_merged)
df_res_merged.reset_index(inplace=True)

In [8]:
# format date fields
df_res_merged['visit_datetime'] = pd.to_datetime(df_res_merged.visit_datetime)
df_res_merged['reserve_datetime'] = pd.to_datetime(df_res_merged.reserve_datetime)

df_res_merged['calendar_date'] = df_res_merged.visit_datetime.dt.date
df_res_merged['visit_time'] = df_res_merged.visit_datetime.dt.time
df_res_merged['reserve_date'] = df_res_merged.reserve_datetime.dt.date
df_res_merged['reserve_time'] = df_res_merged.reserve_datetime.dt.time

In [9]:
# add month, year, and season
df_res_merged['visit_month'] = df_res_merged.visit_datetime.apply(lambda x: x.strftime("%b"))
df_res_merged['visit_year'] = df_res_merged.visit_datetime.apply(lambda x: x.strftime("%Y"))
df_res_merged['reserve_month'] = df_res_merged.reserve_datetime.apply(lambda x: x.strftime("%b"))
df_res_merged['reserve_year'] = df_res_merged.reserve_datetime.apply(lambda x: x.strftime("%Y"))

df_res_merged['reserve_season'] = df_res_merged['reserve_month'].map(seasons)
df_res_merged['visit_season'] = df_res_merged['visit_month'].map(seasons)

In [10]:
# format df_date_info date to merge
df_date_info['calendar_date'] = pd.to_datetime(df_date_info.calendar_date)
df_date_info['calendar_date'] = df_date_info.calendar_date.dt.date
df_res_merged = df_res_merged.merge(df_date_info, on='calendar_date', how='left')
df_res_merged.rename(columns={"day_of_week": "day_of_week_visit", "holiday_flg": "holiday_flag_visit"}, inplace=True)
df_res_merged = df_res_merged.merge(df_date_info, left_on='reserve_date', right_on='calendar_date', how='left')
df_res_merged.rename(columns={"day_of_week": "day_of_week_res", "holiday_flg": "holiday_flag_res","calendar_date_x": "visit_date"}, inplace=True)

In [11]:
# time between reservation and visit
df_res_merged['res_vs_visit'] = df_res_merged['visit_datetime'] - df_res_merged['reserve_datetime']
df_res_merged['res_vs_visit_days'] = df_res_merged['res_vs_visit'].astype('timedelta64[D]')
df_res_merged['res_vs_visit_hours'] = df_res_merged['res_vs_visit'].astype('timedelta64[h]')

In [12]:
# holiday the day before and after visit
df_res_merged['holiday_before_visit'] = df_res_merged.holiday_flag_visit.shift(1)
df_res_merged.holiday_before_visit.fillna(0,inplace=True)
df_res_merged['holiday_after_visit'] = df_res_merged.holiday_flag_visit.shift(-1)
df_res_merged.holiday_after_visit.fillna(0,inplace=True)

In [13]:
# too many genres - amalgamate

df_res_merged.hpg_genre_name.fillna('No Data', inplace=True)
df_res_merged.air_genre_name.fillna('No Data', inplace=True)
df_res_merged.hpg_store_id.fillna('No Data', inplace=True)
df_res_merged.air_store_id.fillna('No Data', inplace=True)
df_res_merged.hpg_area_name.fillna('No Data', inplace=True)
df_res_merged.air_area_name.fillna('No Data', inplace=True)
df_res_merged['air_genre_amal'] = df_res_merged['air_genre_name'].map(genres)
df_res_merged['hpg_genre_amal'] = df_res_merged['hpg_genre_name'].map(genres)

# amalgamated genre - take air genre first then hpg
df_res_merged['genre_amal']=df_res_merged['air_genre_amal']
df_res_merged.loc[df_res_merged['air_genre_amal']=='No Data',['genre_amal']] = df_res_merged['hpg_genre_amal']

In [14]:
# original genre - take air genre first then hpg
df_res_merged['genre_2']=df_res_merged['air_genre_name']
df_res_merged.loc[df_res_merged['air_genre_name']=='No Data',['genre_2']] = df_res_merged['hpg_genre_name']

In [15]:
# area - take air first then hpg
df_res_merged['area_2']=df_res_merged['air_area_name']
df_res_merged.loc[df_res_merged['air_area_name']=='No Data',['area_2']] = df_res_merged['hpg_area_name']

In [16]:
# get prefecture from area
df_res_merged['area_2_pref'] = df_res_merged.area_2.apply(lambda x: x.split(' ')[0] if pd.notnull(x) else x)
df_res_merged.loc[(df_res_merged['area_2_pref']=='No') | (df_res_merged['area_2_pref']=='None'),['area_2_pref']] = 'No Data'

### Wrangle the train/test files

In [17]:
# prep sample submission df
df_sample['visit_date'] = df_sample['id'].map(lambda x: str(x).split('_')[2])
df_sample['air_store_id'] = df_sample['id'].map(lambda x: '_'.join(x.split('_')[:2]))

In [18]:
# append visit and sample file to consistently format features
df_features=df_air_visit.append(df_sample)

In [19]:
# format df_date_info and add month, year, and season

df_features['visit_date'] = pd.to_datetime(df_features.visit_date)
df_features['visit_date'] = df_features.visit_date.dt.date
df_features = df_features.merge(df_date_info, left_on='visit_date', right_on='calendar_date', how='left')
df_features.drop('calendar_date', axis=1, inplace=True)
df_features = df_features.merge(df_air_store,on='air_store_id', how='left')
df_features['visit_month'] = df_features.visit_date.apply(lambda x: x.strftime("%b"))
df_features['visit_year'] = df_features.visit_date.apply(lambda x: x.strftime("%Y"))
df_features['visit_season'] = df_features['visit_month'].map(seasons)

In [20]:
# amalgamate genres like in reservations
df_features.air_genre_name.fillna('No Data', inplace=True)
df_features['air_genre_2'] = df_features['air_genre_name'].map(genres)

In [21]:
# holiday the day before and after visit
df_features['holiday_before_visit'] = df_features.holiday_flg.shift(1)
df_features.holiday_before_visit.fillna(0,inplace=True)
df_features['holiday_after_visit'] = df_features.holiday_flg.shift(-1)
df_features.holiday_after_visit.fillna(0,inplace=True)

In [22]:
# prefecture from area
df_features['area_pref'] = df_features.air_area_name.apply(lambda x: x.split(' ')[0] if pd.notnull(x) else x)

In [23]:
# add golden week
df_features['golden_week'] = df_features.visit_date.apply(lambda x: 1 if 
                            ((x>=(datetime.date(2016,4,29))) & (x<=(datetime.date(2016,5,7)))) |
                            ((x>=(datetime.date(2017,4,29))) & (x<=(datetime.date(2017,5,7))))
                                                                        else 0)

In [24]:
# fix outliers - they dont appear to be an organic spike in visitors
df_features.loc[df_features.visitors>250,['visitors']]= df_features.visitors//10

In [25]:
df_features['weekday'] = df_features.visit_date.apply(lambda x: x.weekday())
df_features['month'] = df_features.visit_date.apply(lambda x: x.month)
df_features['weekend'] = df_features.weekday.apply(lambda x: 1 if x>=5 else 0)
df_features['visit_year'] = df_features.visit_year.astype(int)
df_features['holiday_flg'] = df_features.holiday_flg.astype(float)

In [26]:
# copy original air_visit back to df_air_visit dataframe before calcuating aggregations
df_air_visit = df_features[df_features.id.isna()].copy()

# only include entries after 01 Jul 2016 as this is when we observe a significant and consistent uptick in values
date_incl = datetime.date(2016,7,1)
df_air_visit_incl = df_air_visit[(df_air_visit.visit_date)>=(date_incl)]
df_res_merged_incl = df_res_merged[(df_res_merged.visit_date)>=(date_incl)]

In [27]:
# aggregate df_air_visit dataframe
grp=['air_store_id','air_genre_2','air_genre_name','air_area_name','area_pref']
fld=['visit_month','visit_season','day_of_week','holiday_flg','holiday_before_visit','holiday_after_visit','golden_week','weekend']
aggr={'visitors':['mean','min','max','size']}
col=['v_mean_','v_min_','v_max_','v_cnt']
df_air_visit_grp={}
groupings(df_air_visit_incl,grp,fld,aggr,df_air_visit_grp,col)

In [28]:
# aggregate df_res_merged dataframe
grp=['genre_amal','genre_2','air_store_id','area_2_pref','area_2']
fld=['visit_month','visit_season','day_of_week_visit','holiday_flag_visit','holiday_before_visit','holiday_after_visit','visit_date']
aggr={'res_vs_visit_hours':'mean','reserve_visitors':'size'}
col=['rvvh_','rvs_']
df_res_merged_grp={}
groupings(df_res_merged_incl,grp,fld,aggr,df_res_merged_grp,col)

In [29]:
# add selected aggregates to features df and scale them
vis_grps = ['air_store_id', 'air_store_idvisit_month','air_store_idday_of_week', 
            'air_store_idholiday_flg','air_store_idholiday_after_visit','area_prefday_of_week']

res_grps = ['air_store_id']

for i in df_air_visit_grp:
    if i in vis_grps:
        #scaler = preprocessing.RobustScaler()
        tmp = df_air_visit_grp[i]
        #cols = [col for col in tmp if col.startswith('v_')]
        #tmp[cols] = scaler.fit_transform(tmp[cols])
        df_features = df_features.merge(tmp,how='left')
        
for i in df_res_merged_grp:
    if i in res_grps:
        #scaler = preprocessing.RobustScaler()
        tmp = df_res_merged_grp[i]
        #cols = [col for col in tmp if col.startswith('rv')]
        #tmp[cols] = scaler.fit_transform(tmp[cols])
        df_features = df_features.merge(tmp,how='left')

In [31]:
df_features['lat_plus_long'] = df_features.latitude + df_features.longitude
df_features['lat_sub_max'] = df_features.latitude.max() - df_features.latitude
df_features['long_sub_max'] = df_features.longitude.max() - df_features.longitude
df_features['meanvis_sub_max'] = df_features.v_mean_air_store_id.max() - df_features.v_mean_air_store_id
df_features['meanres_sub_max'] = df_features.rvs_air_store_id.max() - df_features.rvs_air_store_id
df_features['meanvismonth_mul_cnt'] = df_features.v_cntair_store_idvisit_month * df_features.v_mean_air_store_idvisit_month
df_features['meanvisdow_mul_cnt'] = df_features.v_cntair_store_idday_of_week * df_features.v_mean_air_store_idday_of_week
df_features['meanvis_mul_cnt'] = df_features.v_cntair_store_id * df_features.v_mean_air_store_id
df_features['meanvisdow_mul_month'] = df_features.v_mean_air_store_idvisit_month * df_features.v_mean_air_store_idday_of_week
df_features['meanresh_mul_cnt'] = df_features.rvvh_air_store_id * df_features.rvs_air_store_id
df_features['visit_date_int'] = df_features.visit_date.apply(lambda x: x.strftime('%Y%m%d')).astype(int)
df_features['visit_date_int_mul_mean'] = df_features.visit_date_int * df_features.v_mean_air_store_id

In [32]:
# some of the features in the test set have na...work out the mean from the train set to add down below
any_nulls = df_features.isna().any()
any_nulls = list(any_nulls[any_nulls==True].index)
mean_nulls = df_features[df_features.id.notna()].loc[:,any_nulls].mean()

In [33]:
# exclude dummies for:
# - air_store_id as mean per store is unique per store anyway.
# - season as information is included in the month
# - area_name as info included in lat/long
# - genre2 as info in air_genre_name

df_features_dum = pd.get_dummies(df_features[(df_features.visit_date)>=(datetime.date(2016,7,1))],
                                 columns=['air_genre_name','area_pref'],
                                 drop_first=True)

drop_cols = ['id','visit_date','visitors','air_store_id','air_area_name',
             'visit_season','air_genre_2','day_of_week','visit_month','golden_week']

df_train = df_features_dum[df_features_dum.id.isna()].copy()
df_train.drop(columns=drop_cols,inplace=True)

df_test = df_features_dum[df_features_dum.id.notna()].copy()
df_test.drop(columns=drop_cols,inplace=True)

df_test.fillna(mean_nulls,inplace=True)
df_train.fillna(0,inplace=True)

y=np.log1p(df_air_visit_incl['visitors'].values)

In [34]:
# LightGBM
tscv = model_selection.TimeSeriesSplit(n_splits=splits)
tscv_cv = tscv.split(df_train)
params = {  
    "n_estimators": st.randint(10, 100),
    "learning_rate": st.uniform(0.05, 0.4),
    "num_leaves": st.randint(31, 100),
    "max_bin": st.randint(200, 400),
    "min_child_weight": st.randint(1, 10),
    "min_child_samples": st.randint(1, 100),
}   
lgbm = lgb.LGBMRegressor()
lgbmrscv = model_selection.RandomizedSearchCV(lgbm, params, n_iter=10)  
lgbmrscv.fit(df_train, y)
print(lgbmrscv.best_params_)
print('RMSE LGBMRegressor: ', RMSLE(y, lgbmrscv.predict(df_train)))
df_sample['visitors'] = lgbmrscv.predict(df_test)
df_sample['visitors'] = np.expm1(df_sample['visitors']).clip(lower=1.)
lgbmrscv_out = df_sample[['id','visitors']].copy()
lgbmrscv_out.to_csv('lgbmrscv_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)

{'learning_rate': 0.095800265773813162, 'max_bin': 243, 'min_child_samples': 73, 'min_child_weight': 1, 'n_estimators': 89, 'num_leaves': 93}
RMSE LGBMRegressor:  0.450567620375


In [35]:
# LightGBM with Time Series CV
tscv = model_selection.TimeSeriesSplit(n_splits=splits)
tscv_cv = tscv.split(df_train)
y_test_pred = 0
lgbmtscv = lgbmrscv.best_estimator_
for i, (train_index, test_index) in enumerate(tscv.split(df_train)):
    # Create data for this fold
    y_train, y_valid = y[train_index].copy(), y[test_index]
    X_train, X_valid = df_train.iloc[train_index, :].copy(), df_train.iloc[test_index, :].copy()
    print("\nFold ", i)

    fit_model = lgbmtscv.fit(X_train, y_train)
    pred = lgbmtscv.predict(X_valid)
    print('RMSE LightGBM, fold ', i, ': ', RMSLE(y_valid, pred))
    print('Prediction length on validation set, LightGBM, fold ', i, ': ', len(pred))
    # Accumulate test set predictions

    pred = lgbmtscv.predict(df_test)
    print('Prediction length on test set, LightGBM, fold ', i, ': ', len(pred))
    y_test_pred += pred

y_test_pred/=splits
df_sample['visitors'] = y_test_pred
df_sample['visitors'] = np.expm1(df_sample['visitors']).clip(lower=1.)
lgbmtscv_out = df_sample[['id','visitors']].copy()
lgbmtscv_out.to_csv('lgbmtscv_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)


Fold  0
RMSE LightGBM, fold  0 :  0.480110703579
Prediction length on validation set, LightGBM, fold  0 :  51102
Prediction length on test set, LightGBM, fold  0 :  32019

Fold  1
RMSE LightGBM, fold  1 :  0.451070430976
Prediction length on validation set, LightGBM, fold  1 :  51102
Prediction length on test set, LightGBM, fold  1 :  32019

Fold  2
RMSE LightGBM, fold  2 :  0.448351649787
Prediction length on validation set, LightGBM, fold  2 :  51102
Prediction length on test set, LightGBM, fold  2 :  32019


In [36]:
# XGBRegressor
tscv = model_selection.TimeSeriesSplit(n_splits=splits)
tscv_cv = tscv.split(df_train)
params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}

xgbr = xgb.XGBRegressor()  
xgbrscv = model_selection.RandomizedSearchCV(xgbr, params, n_iter=10)  
xgbrscv.fit(df_train, y)  
print('RMSE XGBRegressor: ', RMSLE(y, xgbrscv.predict(df_train)))
df_sample['visitors'] = xgbrscv.predict(df_test)
df_sample['visitors'] = np.expm1(df_sample['visitors']).clip(lower=1.)
xgbrscv_out = df_sample[['id','visitors']].copy()
xgbrscv_out.to_csv('xgbrscv_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)

RMSE XGBRegressor:  0.453625769871


In [37]:
# XGBRegressor with Time Series CV
tscv = model_selection.TimeSeriesSplit(n_splits=splits)
tscv_cv = tscv.split(df_train)
y_test_pred = 0
xgbtscv = xgbrscv.best_estimator_
for i, (train_index, test_index) in enumerate(tscv.split(df_train)):
    # Create data for this fold
    y_train, y_valid = y[train_index].copy(), y[test_index]
    X_train, X_valid = df_train.iloc[train_index, :].copy(), df_train.iloc[test_index, :].copy()
    print("\nFold ", i)

    fit_model = xgbtscv.fit(X_train, y_train)
    pred = xgbtscv.predict(X_valid)
    print('RMSE XGBRegressor, fold ', i, ': ', RMSLE(y_valid, pred))
    print('Prediction length on validation set, XGBRegressor, fold ', i, ': ', len(pred))
    # Accumulate test set predictions

    pred = xgbtscv.predict(df_test)
    print('Prediction length on test set, XGBRegressor, fold ', i, ': ', len(pred))
    y_test_pred += pred

y_test_pred/=splits
df_sample['visitors'] = y_test_pred
df_sample['visitors'] = np.expm1(df_sample['visitors']).clip(lower=1.)
xgbtscv_out = df_sample[['id','visitors']].copy()
xgbtscv_out.to_csv('xgbtscv_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)


Fold  0
RMSE XGBRegressor, fold  0 :  0.486584679377
Prediction length on validation set, XGBRegressor, fold  0 :  51102
Prediction length on test set, XGBRegressor, fold  0 :  32019

Fold  1
RMSE XGBRegressor, fold  1 :  0.458001003219
Prediction length on validation set, XGBRegressor, fold  1 :  51102
Prediction length on test set, XGBRegressor, fold  1 :  32019

Fold  2
RMSE XGBRegressor, fold  2 :  0.454047527341
Prediction length on validation set, XGBRegressor, fold  2 :  51102
Prediction length on test set, XGBRegressor, fold  2 :  32019


In [38]:
sub=pd.DataFrame()
sub=xgbtscv_out.merge(xgbrscv_out, on='id',suffixes=('_xgbtscv','_xgbrscv'))\
               .merge(lgbmtscv_out,on='id')\
               .merge(lgbmrscv_out,on='id',suffixes=('lgbmtscv','lgbmrscv'))
df_sample['visitors'] = sub.mean(axis=1)
mean_out = df_sample[['id','visitors']].copy()
mean_out.to_csv('mean_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)

In [43]:
sub=pd.DataFrame()
sub=lgbmtscv_out.merge(lgbmrscv_out,on='id',suffixes=('lgbmtscv','lgbmrscv'))
df_sample['visitors'] = sub.mean(axis=1)
mean_out = df_sample[['id','visitors']].copy()
mean_out.to_csv('mean_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)

In [42]:
a = xgbrscv.best_estimator_.booster().get_fscore()
b = pd.DataFrame(list(a.values()),index=list(a.keys())).sort_values(by=0,ascending=False)
c = pd.DataFrame(lgbmrscv.best_estimator_.booster_.feature_importance(),
             lgbmrscv.best_estimator_.booster_.feature_name()).sort_values(by=0,ascending=False)
c.merge(b,how='left',left_index=True,right_index=True).head(50)

,0_x,0_y
visit_date_int,879,343.0
meanvisdow_mul_month,723,240.0
v_mean_air_store_id,465,201.0
v_max_air_store_idday_of_week,364,73.0
v_max_air_store_idvisit_month,355,86.0
meanvisdow_mul_cnt,321,80.0
v_min_air_store_idvisit_month,319,84.0
v_mean_air_store_idholiday_flg,295,62.0
v_min_air_store_idday_of_week,278,55.0
meanvismonth_mul_cnt,253,68.0


In [44]:
sub

,id,visitorslgbmtscv,visitorslgbmrscv
0,air_00a91d42b08b08d9_2017-04-23,3.772596,2.621303
1,air_00a91d42b08b08d9_2017-04-24,22.581868,22.066151
2,air_00a91d42b08b08d9_2017-04-25,24.432921,25.464372
3,air_00a91d42b08b08d9_2017-04-26,30.617921,30.625976
4,air_00a91d42b08b08d9_2017-04-27,31.431971,31.452225
5,air_00a91d42b08b08d9_2017-04-28,41.464704,41.201001
6,air_00a91d42b08b08d9_2017-04-29,19.809236,20.262384
7,air_00a91d42b08b08d9_2017-04-30,3.741127,2.796661
8,air_00a91d42b08b08d9_2017-05-01,19.946098,19.852723
9,air_00a91d42b08b08d9_2017-05-02,28.764996,31.607172
